In [27]:
import torch
import numpy as np
torch.manual_seed(0)

The point of this notebook is to better understand the reparamterization trick which is really one of the biggest tricks in the VAE. Per the point made [here](https://gregorygundersen.com/blog/2018/04/29/reparameterization/) this notebook demonstrates that torch can "differentiate through random nodes" and it even does it fine, the real necessity for the reparam trick is that when your distribution and function depend on the same parameters you get a product rule in the expectation. Unless you can evaluate things analytically like in my toy example, you will always get a nonsensical estimate for the gradient of this expectation using auto grad.

First we compute an expectation that torch can handle
$$\nabla_\theta \mathbb{E}_{p(z)}[f_\theta(z)] = \int_z p(z) \nabla_\theta f_\theta(z) dz = \mathbb{E}_{p(z)}[\nabla_\theta f_\theta(z)]$$
**the key thing here is that this is only true when $p_{(z)}$ does not also depend on $\theta$**

Now define
$$p_(z) = N(z|1,0)$$
$$f_\theta(z) = 3\theta z^2$$

We can compute these expectations analytically and see they should be 3, next we compute them two ways $\nabla_\theta \mathbb{E}_{p(z)}[f_\theta(z)], \mathbb{E}_{p(z)}[\nabla_\theta f_\theta(z)]$ the first using autograd in pytorch the second by giving the gradient function to monte carlo.

In [149]:
def f(t,z):
    return 3*t*z**2

def grad_f(z):
    return 3*z**2

N = 10000
xs = torch.randn(N)
t = torch.Tensor([3])
t.requires_grad = True
y = 3*t*xs**2
y = torch.mean(y)
y.backward()
print(f"torch: grad E[f] = {t.grad.item()}")

o_xs = np.random.randn(N)
print(f"MC: E[grad f] = {np.mean(grad_f(o_xs))}")

torch: grad E[f] = 3.007117748260498
MC: E[grad f] = 2.9939215657133142


Now we instead let $p(z)$ depend on theta
$$p_\theta(z) = N(z|\theta,0)$$
$$f_\theta(z) = 3\theta z^2$$

This gives a totally different equation for the gradient of the expectation
$$\nabla_\theta \mathbb{E}_{p_\theta(z)}[f_\theta(z)] = \int_z f_\theta(z)\nabla_\theta p_\theta(z) dz + \mathbb{E}[\nabla_\theta f_\theta(z)]$$



$$\int_z f_\theta(z)\nabla_\theta p_\theta(z) dz + \int_z \nabla_\theta f_\theta(z) p_\theta(z) dz = -6t^2 + 30$$

In [134]:
N = 1000000

theta = 3
t = torch.Tensor([theta])
t.requires_grad = True
xs = torch.normal(mean=t.repeat(1,N).squeeze(),std=torch.ones(N))
y = 3*t*xs**2
y = torch.mean(y)
y.backward()


print(f"Analyatic grad E_t[f] = {-6*theta**2 + 30}")
o_xs = np.random.normal(loc=theta,size=N)
print(f"MC E[grad f] = {-6*theta**2 + np.mean(grad_f(o_xs))}")
print(f"Autograd E_t[f] = {t.grad.item()}")

Analyatic grad E_t[f] = -24
MC E[grad f] = -23.982790434748306
Autograd E_t[f] = 30.004558563232422


We can see that the autograd will **never** get close to estimating the expectation, because it simply isnt dealing with the right quantity anymore.
This gets at the heart of the real point of the paramterization trick, if $g_\theta$ is differentiable then we define 
$$\epsilon \sim p(\epsilon)$$
$$z = g_\theta(\epsilon,x)$$
Since $g$ is deterministic in $x$ we can swap out the distribution in our expectation for $p(\epsilon)$ which no longer depends on $\theta$
$$\nabla_\theta \mathbb{E}_{p_\theta(z)}[f(z)] = \nabla_\theta \mathbb{E}_{p_\epsilon}[f(g_\theta(\epsilon,x))] =  \mathbb{E}_{p_\epsilon}[\nabla_\theta f(g_\theta(\epsilon,x))]$$

as shown [here](https://nbviewer.org/github/gokererdogan/Notebooks/blob/master/Reparameterization%20Trick.ipynb) this not only makes it so we can use backprop, this estimator also seems to have nicer statistical properties like less variance.